In [1]:
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras import callbacks

 
# Create the model
# Create the model
model =models.Sequential()

model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (128,128,3)))
model.add(layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))


model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))


model.add(layers.Flatten())
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(6, activation = "softmax"))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      2432      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)            

In [2]:
import os
base_dir = '/kaggle/input/duth-cv-2019-2020-hw-4/vehicles'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=30,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest")
       
val_datagen  = ImageDataGenerator(rescale=1./255,)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    # color_mode='grayscale',
                                                    target_size=(128,128),
                                                    shuffle=True)     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  val_datagen.flow_from_directory(validation_dir,
                                                        batch_size=64,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(128,128)) 











Found 2494 images belonging to 6 classes.
Found 311 images belonging to 6 classes.


In [4]:

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.0001),
              metrics=['acc'])
# Train the model
call = callbacks.ModelCheckpoint("best.h5", monitor='val_loss', mode = 'min', verbose=1, save_best_only = True)
early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=60,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1,callbacks=[call,early])
 
# Save the model


Epoch 1/60
39/38 [==============================] - 28s 722ms/step - loss: 1.6711 - acc: 0.3083 - val_loss: 1.5469 - val_acc: 0.4148

Epoch 00001: val_loss improved from inf to 1.54689, saving model to best.h5
Epoch 2/60
39/38 [==============================] - 23s 578ms/step - loss: 1.5060 - acc: 0.3966 - val_loss: 1.4604 - val_acc: 0.3730

Epoch 00002: val_loss improved from 1.54689 to 1.46039, saving model to best.h5
Epoch 3/60
39/38 [==============================] - 22s 572ms/step - loss: 1.4499 - acc: 0.4294 - val_loss: 1.3709 - val_acc: 0.4437

Epoch 00003: val_loss improved from 1.46039 to 1.37095, saving model to best.h5
Epoch 4/60
39/38 [==============================] - 24s 603ms/step - loss: 1.3875 - acc: 0.4551 - val_loss: 1.2113 - val_acc: 0.5080

Epoch 00004: val_loss improved from 1.37095 to 1.21133, saving model to best.h5
Epoch 5/60
39/38 [==============================] - 22s 571ms/step - loss: 1.3473 - acc: 0.4703 - val_loss: 1.5331 - val_acc: 0.4759

Epoch 00005: v

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from keras.preprocessing import image
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import csv

model = tf.keras.models.load_model('/kaggle/working/best.h5')
rowlist = [['Id', 'Category']]

for dirname, _, filenames in os.walk('/kaggle/input/duth-cv-2019-2020-hw-4/vehicles/test'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        img = image.load_img(path, target_size=(128, 128), grayscale=False, interpolation='bilinear')
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        classes_pred = model.predict(x)
        cls_pred = np.argmax(classes_pred)
        rowlist.append([filename, cls_pred])
        #print(filename, cls_pred)
        with open('output.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rowlist)
        